## Ball in a Box

Use Euler's algorithm to solve the ODEs representing the motion of a ball in a box. Discussion of this problem is found on pages 36 anfd 37 of Gould. *Warning* Gould spends most of Chapter 2 trying to teach the student to object oriented programming and Java. We are not interested in that in this class. You can ignore those portions and look forward to later chapters that do less in this regard.

This unit is directed at two difficulties students have in this class:

1. writing ODEs of several variables. Too many examples are one variable. Here, we *independently* evolve equations for $x$ and $y$ coordinates while keeping track of the velocity components $v_x$ and $v_y$.
2. animation of output is a powerful way of troubleshooting many of the assignments in this class, yet, implementation has always been a bugaboo. Here, we start on getting some animation in our output.

#### Algorithm  
Understanding that the $x$ and $y$ components can be treated independently is expressed by independent equations for the $x$ and $y$ coordinates. The result of applying Euler's algorithm to the equations

$$\frac{dx}{dt} = v_x$$
$$\frac{dy}{dt} = v_y$$

and done on the board is:

$$x(t+\Delta t) = x(t) + v_x(t) \Delta t$$
$$y(t+\Delta t) = y(t) - v_y(t) \Delta t$$

The basis for applying the algorithm is:

1. update positions using Euler's algoritm. 
2. check to see if the ball has exited the box. Let's say the box goes from -1 to 1 in both $x$ and $y$ directions. If it leaves the box in the $x$ direction then flip the sign of $v_y$. If it leaves inthe $y$ direction, flip the sign of $v_x$.
3. make updates to the display.

#### Implementation
The second problem involves getting the updates to occur in conjuntion with animation. This an be done using matplotlibs `animation` class, but is a little tricky. The problem of dynamically updating simulation results occurs frequently in this class, so I am trying to get students to learn the technique early. Below is a simple sketch of the concept. While this works, it still leaves something to be desired. I'll continue to work on this and pass tips along to you.


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation
from matplotlib import rc
import numpy as np

# equivalent to rcParams['animation.html'] = 'html5'
# This line sets the stage for animation.
rc('animation', html='html5')

# A simple function to animate
# note the linspace function. Useful!
t = np.linspace(0,2*np.pi,50)
x = np.sin(t)

# Create a figure ,fig of a simple axes running from -1 to 1 in y.
# Retain the axis handle, ax, and the plot handle, l
# for later use in creating the plot (axis handle) and
# manupulating contents (plot handle).
fig, ax = plt.subplots()
l, = ax.plot([0,2*np.pi],[-1,1])

# This is the function that performs the animation. In this case all it does is
# move the coordinates of the point being plotted, while maintaining the history
# with the indexing [:i]
animate = lambda i: l.set_data(t[:i], x[:i])

# Create the animation object that includes the callback function animate, the figure handle fig
# and the number of frames to show.
# side effect - this creates an empty axes.
ani = matplotlib.animation.FuncAnimation(fig, animate, frames=len(t));

# create the animation by 'displaying' the object
ani

#### Algorithm  
Understanding that the $x$ and $y$ components can be treated independently is expressed by independent equations for the $x$ and $y$ coordinates. The result of applying Euler's algorithm to the equations

$$\frac{dx}{dt} = v_x$$
$$\frac{dy}{dt} = v_y$$

and done on the board is:

$$x(t+\Delta t) = x(t) + v_x(t) \Delta t$$
$$y(t+\Delta t) = y(t) - v_y(t) \Delta t$$

The basis for applying the algorithm is:

1. update positions using Euler's algoritm. 
2. check to see if the ball has exited the box. Let's say the box goes from -1 to 1 in both $x$ and $y$ directions. If it leaves the box in the $x$ direction then flip the sign of $v_y$. If it leaves inthe $y$ direction, flip the sign of $v_x$.
3. make updates to the display.

In [ ]:
box_half = 1
box = [-box_half,box_half]
dt = 0.01
t = 0
x = 0.5
y = 0
vx = 1
vy = 1
fig, ax = plt.subplots()
l, = ax.plot([0],[0],'ro')
ax.set_xlim(box)
ax.set_ylim(box)

def animate(i):
    global x,y,vx,vy,t
    x += vx*dt # x position update
    y += vy*dt # y position update
    t += dt   # time update
    if x <= box[0] or x >= box[1]: # swap y velocity if ball leaves box in y direction
        vx = -vx
    if y <= box[0] or y >= box[1]: # swap x velocity if ball leaves box in x direction
        vy = -vy
    l.set_data([x],[y])      # update the plot
    return l

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=1000, interval=20);
ani

In [ ]:
box_half = 1
box = [-box_half,box_half]
dt = 0.01
t = 0
x = 0.5
y = 0
vx = 1
vy = 1
g = -9.8
fig, ax = plt.subplots()
l, = ax.plot([0],[0],'ro')
ax.set_xlim(box)
ax.set_ylim(box)

def animate(i):
    global x,y,vx,vy,t
    if x <= box[0] or x >= box[1]: # swap y velocity if ball leaves box in y direction
        vx = -vx
    if y <= box[0] or y >= box[1]: # swap x velocity if ball leaves box in x direction
        vy = -vy
    x += vx*dt # x position update
    y += vy*dt # y position update
    vy += g*dt # gravity
    t += dt   # time update
    
    l.set_data([x],[y])      # update the plot
    return l

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=1000, interval=20);
ani

In [ ]:
import pygame
import sys
import math

pygame.init()

WIDTH = 800
HEIGHT = 800
FPS = 60
DT = 0.01

WHITE = (255, 255, 255)
RED = (255, 0, 0)
GRAY = (100, 100, 100)
BLACK = (0, 0, 0)

g = 0#980
balls = []

# Ball sizes
BALL_SIZES = {
    "small": {"radius": 7, "mass": 0.5},
    "normal": {"radius": 10, "mass": 1.0},
    "large": {"radius": 15, "mass": 2.0}
}

class Ball:
    def __init__(self, x, y, vx, vy, size):
        self.x = x
        self.y = y
        self.vx = vx
        self.vy = vy
        self.size = size
        self.radius = BALL_SIZES[size]["radius"]
        self.mass = BALL_SIZES[size]["mass"]

    def update(self):
        if self.x <= 0 or self.x >= WIDTH:
            self.vx = -self.vx
        if self.y <= 0 or self.y >= HEIGHT:
            self.vy = -self.vy
        
        self.x += self.vx * DT
        self.y += self.vy * DT
        self.vy += g * DT

    def check_collision(self, other):
        dx = self.x - other.x
        dy = self.y - other.y
        distance = math.sqrt(dx*dx + dy*dy)
        
        if distance < self.radius + other.radius:
            normal_x = dx / distance
            normal_y = dy / distance
            
            relative_velocity_x = self.vx - other.vx
            relative_velocity_y = self.vy - other.vy
            
            relative_velocity_dot_normal = (relative_velocity_x * normal_x + 
                                         relative_velocity_y * normal_y)
            
            impulse = 2.0 * relative_velocity_dot_normal / (self.mass + other.mass)
            
            self.vx -= impulse * other.mass * normal_x
            self.vy -= impulse * other.mass * normal_y
            other.vx += impulse * self.mass * normal_x
            other.vy += impulse * self.mass * normal_y
            
            overlap = (self.radius + other.radius) - distance
            self.x += overlap * normal_x / 2
            self.y += overlap * normal_y / 2
            other.x -= overlap * normal_x / 2
            other.y -= overlap * normal_y / 2

screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Click and Drag to Launch")
clock = pygame.time.Clock()
font = pygame.font.SysFont(None, 36)

dragging = False
drag_start = None
power_multiplier = 2.0
current_size = "normal"

while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        elif event.type == pygame.MOUSEBUTTONDOWN:
            dragging = True
            drag_start = pygame.mouse.get_pos()
        elif event.type == pygame.MOUSEBUTTONUP and dragging:
            end_pos = pygame.mouse.get_pos()
            dx = drag_start[0] - end_pos[0]
            dy = drag_start[1] - end_pos[1]
            
            distance = math.sqrt(dx*dx + dy*dy)
            vx = dx * power_multiplier
            vy = dy * power_multiplier
            
            balls.append(Ball(drag_start[0], drag_start[1], vx, vy, current_size))
            dragging = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_1:
                current_size = "small"
            elif event.key == pygame.K_2:
                current_size = "normal"
            elif event.key == pygame.K_3:
                current_size = "large"
    
    screen.fill(WHITE)
    
    for ball in balls:
        ball.update()
    
    for i in range(len(balls)):
        for j in range(i + 1, len(balls)):
            balls[i].check_collision(balls[j])
    
    for ball in balls:
        pygame.draw.circle(screen, RED, (int(ball.x), int(ball.y)), ball.radius)
    
    if dragging:
        current_pos = pygame.mouse.get_pos()
        dx = drag_start[0] - current_pos[0]
        dy = drag_start[1] - current_pos[1]
        distance = math.sqrt(dx*dx + dy*dy)
        
        pygame.draw.line(screen, GRAY, drag_start, current_pos, 2)
        indicator_radius = min(distance * 0.2, 50)
        pygame.draw.circle(screen, GRAY, drag_start, int(indicator_radius), 2)
    
    # Draw current size indicator
    size_text = f"Ball Size: {current_size.capitalize()}"
    text_surface = font.render(size_text, True, BLACK)
    screen.blit(text_surface, (10, 10))
    
    pygame.display.flip()
    clock.tick(FPS)